# Direct and inverse geometry of 3d robots
This notebook introduces the kinematic tree of Pinocchio for a serial manipulator, explain how to compute the forward and inverse geometry (from configuration to end-effector placements, and inversely). The ideas are examplified with a simplified case-study taken from parallel robotics.

In [1]:
import gepetuto.magic

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide
    the solutions to some questions. Change it for %load if you want to see (and
    execute) the solution.


## Set up
We will need Pinocchio, Gepetto-Viewer, SciPy for the solvers

In [2]:
import time
import math
import numpy as np
from numpy.linalg import norm
import pinocchio as pin
import example_robot_data as robex
from scipy.optimize import fmin_bfgs

## Kinematic tree in Pinocchio
Let's now play with 3D robots. We will load the models from URDF files.

*The robot UR5* is a low-cost manipulator robot with good performances. It is a fixed robot with one 6-DOF arms developed by the Danish company Universal Robot. All its 6 joints are revolute joints. Its configuration is in R^6 and is not subject to any constraint. The model of UR5 is described in a URDF file, with the visuals of the bodies of the robot being described as meshed (i.e. polygon soups) using the Collada format ".dae". Both the URDF and the DAE files are available in the repository in the model directory. 

This robot model, as well as other models used in the notebooks, are installed from the apt paquet robotpkg-example-robot-data and stored in /opt/openrobots/share/example-robot-data.


In [3]:
# %load tp2/generated/simple_pick_and_place_1

robot = robex.load('ur5')


The kinematic tree is represented by two C++ objects called Model (which contains the model constants: lengths, masses, names, etc) and Data (which contains the working memory used by the model algorithms). Both C\++ objects are contained in a unique Python class. The first class is called RobotWrapper and is generic.

In [4]:
print(robot.model)

Nb joints = 7 (nq=6,nv=6)
  Joint 0 universe: parent=0
  Joint 1 shoulder_pan_joint: parent=0
  Joint 2 shoulder_lift_joint: parent=1
  Joint 3 elbow_joint: parent=2
  Joint 4 wrist_1_joint: parent=3
  Joint 5 wrist_2_joint: parent=4
  Joint 6 wrist_3_joint: parent=5



For the next steps, we are going to work with the RobotWrapper.

Import the class RobotWrapper and create an instance of this class in the python terminal. At initialization, RobotWrapper will read the model description in the URDF file given as argument. In the following, we will use the model of the UR5 robot, available in the directory "models" of pinocchio (available in the homedir of the VBox). The code of the RobotWrapper class is in /opt/openrobots/lib/python2.7/site-packages/pinocchio/robot_wrapper.py . Do not hesitate to have a look at it and to take inspiration from the implementation of the class functions.

Here are some import methods of the class.


* robot.q0 contains a reference initial configuration of the robot (not a pretty good one for the UR-5).

* robot.index('joint name') returns the index of the joint.

In [5]:
robot.index(' wrist_3_joint')

7

* robot.model.names is a container (~list) that contains all the joint names

In [6]:
for i, n in enumerate(robot.model.names):
    print(i, n)

0 universe
1 shoulder_pan_joint
2 shoulder_lift_joint
3 elbow_joint
4 wrist_1_joint
5 wrist_2_joint
6 wrist_3_joint


* robot.model.frames contains all the import frames attached to the robot. 

In [7]:
for f in robot.model.frames:
    print(f.name, 'attached to joint #', f.parent)

universe attached to joint # 0
world attached to joint # 0
world_joint attached to joint # 0
base_link attached to joint # 0
base_link-base_fixed_joint attached to joint # 0
base attached to joint # 0
shoulder_pan_joint attached to joint # 1
shoulder_link attached to joint # 1
shoulder_lift_joint attached to joint # 2
upper_arm_link attached to joint # 2
elbow_joint attached to joint # 3
forearm_link attached to joint # 3
wrist_1_joint attached to joint # 4
wrist_1_link attached to joint # 4
wrist_2_joint attached to joint # 5
wrist_2_link attached to joint # 5
wrist_3_joint attached to joint # 6
wrist_3_link attached to joint # 6
ee_fixed_joint attached to joint # 6
ee_link attached to joint # 6
wrist_3_link-tool0_fixed_joint attached to joint # 6
tool0 attached to joint # 6


/tmp/ipykernel_72435/2471843488.py:2: UserWarning: Deprecated member. Use Frame.parentJoint instead.
  print(f.name, 'attached to joint #', f.parent)


* robot.placement(idx) and robot.framePlacement(idx) returns the placement (i.e. translation+rotation of the joint / frame in argument.

In [8]:
robot.placement(robot.q0, 6)  # Placement of the end effector.

SE3(array([[-1.0000000e+00,  0.0000000e+00,  9.7932773e-12,  8.1725000e-01],[ 0.0000000e+00,  1.0000000e+00,  0.0000000e+00,  1.0915000e-01],[-9.7932773e-12,  0.0000000e+00, -1.0000000e+00, -5.4910000e-03],[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]]))

The dimension of the configuration space (i.e. the number of joints) is given in:

In [9]:
NQ = robot.model.nq
NV = robot.model.nv  # for this simple robot, NV == NQ

## Display simple geometries
The robot is displayed in the viewer. We are going to use Meshcat to visualize the 3d robot and scene. First open the viewer and load the robot geometries.

In [11]:
from supaero2025.meshcat_viewer_wrapper import MeshcatVisualizer, colors  # noqa: E402

viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [12]:
viz.viewer.jupyter_cell()

A configuration *q* can be displayed in the viewer:

In [14]:
q = np.array([-1., -1.5, 2.1, -.5, -.5, 0])

viz.display(q)

Other geometries (cubes, spheres, etc) can be displayed as well.

In [27]:
# %load tp2/generated/simple_pick_and_place_2

# Add a red box in the viewer
ballID = "world/ball"
viz.addSphere(ballID, 0.1, colors.red)

# Place the ball at the position ( 0.5, 0.1, 0.2 )
# The viewer expect position and rotation, apppend the identity quaternion
q_ball = [0.5, 0.1, 0.2, 1, 0, 0, 0]
viz.applyConfiguration(ballID, q_ball)


# Forward (direct) geometry

First, let's do some forward geometry, i.e. use Pinocchio to compute where is the end effector knowning the robot configuration.

# Simple pick ...

Say we have a target at position [.5,.1,.2] and we would like the robot to grasp it.
First decide (by any way you want, e.g. trial and error) the configuration of the robot so that the end effector touches the ball. For that, modify the template code below.


In [17]:
q0 = np.zeros(NQ)  # set the correct values here
q0[0] = 0.5
q0[1] = 0.
q0[2] = -1.5
q0[3] = 0.
q0[4] = 0.
q0[5] = 0.

viz.display(q0)

# Take care to explicitely mention copy when you want a copy of array.
q = q0.copy()

Here is the solution, should you need it.

In [25]:
# %load tp2/generated/simple_pick_and_place_3
q0 = np.zeros(NQ)  # set the correct values here
q0[0] = -0.375
q0[1] = -1.2
q0[2] = 1.71
q0[3] = -q0[1] - q0[2]
q0[4] = q0[0]
q0[5] = 0.0

viz.display(q0)
q = q0.copy()


# ... and simple place

At the reference position you built, the end effector placement can be obtained by calling 

In [28]:
robot.placement(q, 6).translation

array([ 0.50182315, -0.08022936,  0.19913809])

Only the translation part of the placement has been selected. The rotation is free.

Now, choose any trajectory you want in the configuration space (it can be sinus-cosinus waves, polynomials, splines, straight lines). Make a for loop to display the robot at sampling positions along this trajectory. The function sleep can be used to slow down the loop.

In [21]:
time.sleep(.1)  # in second

At each instant of your loop, recompute the position of the ball and display it so that it always "sticks" to the robot end effector, by modifying the template code below:

The solution is below, should you need it.

In [29]:
# %load tp2/generated/simple_pick_and_place_4
# Random velocity of the robot driving the movement

vq = np.array([2.0, 0, 0, 4.0, 0, 0])
q = q0.copy()
viz.display(q)

idx = robot.index("wrist_3_joint")
o_eff = robot.placement(
    q, idx
).translation  # Position of end-eff wrt world at current configuration
o_ball = q_ball[:3]  # Position of ball wrt world
eff_ball = o_ball - o_eff  # Position of ball wrt eff

for i in range(50):
    # Chose new configuration of the robot
    q += vq / 40
    q[2] = 1.71 + math.sin(i * 0.05) / 2

    # Gets the new position of the ball
    o_ball = robot.placement(q, idx) * eff_ball

    # Display new configuration for robot and ball
    viz.applyConfiguration(ballID, o_ball.tolist() + [1, 0, 0, 0])
    viz.display(q)
    time.sleep(1e-1)


### Pick and place in 3D

Say now that the object is a rectangle and not a sphere. Pick the object at a reference position with the rotation that is imposed, so that the end effector is aligned with one of the faces of the rectangle.


In [36]:
# %load tp2/generated/simple_pick_and_place_5

# Add a red box in the viewer
boxID = "world/box"
viz.delete(ballID)
viz.addBox(boxID, [0.1, 0.2, 0.1], colors.magenta)

# Place the box at the position (0.5, 0.1, 0.2)
q_box = [0.5, 0.1, 0.2, 1, 0, 0, 0]
viz.applyConfiguration(boxID, q_box)
viz.applyConfiguration(ballID, [2,2,2,1,0,0,0])


A configuration with the arm nicely attached to the box is:

In [37]:
# %load tp2/generated/simple_pick_and_place_6

q0 = np.zeros(NQ)
q0[0] = -0.375
q0[1] = -1.2
q0[2] = 1.71
q0[3] = -q0[1] - q0[2]
q0[4] = q0[0]

viz.display(q0)
q = q0.copy()


Redo the same question as before, but now also choosing the orientation of the box. For that, at each robot configuration in your for-loop, compute the box placement wrt the world (let's denote it by oMbox) and display both the box and the robot configuration in the view.

In [39]:
# %load tp2/generated/simple_pick_and_place_7
# Random velocity of the robot driving the movement
vq = np.array([2., 0, 0, 4., 0, 0])

idx = robot.index('wrist_3_joint')
oMeff = robot.placement(q, idx)  # Placement of end-eff wrt world at current configuration
oMbox = pin.XYZQUATToSE3(q_box)  # Placement of box     wrt world
effMbox = oMeff.inverse() * oMbox  # Placement of box     wrt eff

for i in range(100):
    # Chose new configuration of the robot
    q += vq / 40
    q[2] = 1.71 + math.sin(i * 0.05) / 2

    # Gets the new position of the box
    oMbox = robot.placement(q, idx) * effMbox

    # Display new configuration for robot and box
    viz.applyConfiguration(boxID, oMbox)
    viz.display(q)
    time.sleep(1e-2)


## Inverse geometry

We only yet computed the forward geometry, i.e. from configurations to end-effector placement. Let's to the inverse map not.

### Inverse geometry in 3D

Let's now first control the position (i.e. translation only) of the end effector of a manipulator robot to a given position. For this first part, we will use the fixed serial-chain robot model.

Recall first that the position (3D) of the joint with index "i=6" at position "q" can be access by the following two lines of code.

In [24]:
robot.placement(q, 6).translation

Using the scipy solver [used in the previous notebook](1_geometry_2d.ipynb#section_optim), compute a configuration q where the end effector reaches p. For that, implement a cost function that takes a configuration as argument and returns the squared distance between the end effetor and the target.

In [45]:
# %load tp2/generated/invgeom3d_1
# Add a vizualisation for the tip of the arm.
viz.delete(boxID)
tipID = "world/blue"
viz.addBox(tipID, [0.08] * 3, [0.2, 0.2, 1.0, 0.5])
#
# OPTIM 3D #########################################################
#


def cost(q):
    """Compute score from a configuration"""
    p = robot.placement(q, 6).translation
    return norm(p - target) ** 2


def callback(q):
    viz.applyConfiguration(ballID, target.tolist() + [0, 1, 0, 0])
    viz.applyConfiguration(tipID, robot.placement(q, 6))
    viz.display(q)
    time.sleep(1)


target = np.array([0.5, 0.1, 0.2])  # x,y,z
qopt = fmin_bfgs(cost, robot.q0, callback=callback)


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 17
         Function evaluations: 147
         Gradient evaluations: 21


### Inverse geometry in 6D
6D means: translation and rotation. Change the previous cost function for a cost measuring the difference between the current placement root.placement(q,6) and a reference placement oMdes. 
For that, you can use the SE(3) log function to score the distance between two placements. The log returns a 6D velocity, represented by a class Motion, that must be transformed to a vector of R^6 from which you can take the norm.


In [26]:
pin.log(pin.SE3.Identity()).vector

In [46]:
# %load tp2/generated/invgeom6d_1

# Add a vizualisation for the tip of the arm.
tipID = "world/blue"
viz.addBox(tipID, [.08] * 3, [.2, .2, 1., .5])

#
# OPTIM 6D #########################################################
#


def cost(q):
    '''Compute score from a configuration'''
    M = robot.placement(q, 6)
    return norm(pin.log(M.inverse() * Mtarget).vector)


def callback(q):
    viz.applyConfiguration(boxID, Mtarget)
    viz.applyConfiguration(tipID, robot.placement(q, 6))
    viz.display(q)
    time.sleep(1e-2)


Mtarget = pin.SE3(pin.utils.rotate('x', 3.14 / 4), np.array([-0.5, 0.1, 0.2]))  # x,y,z
qopt = fmin_bfgs(cost, robot.q0, callback=callback)

print('The robot finally reached effector placement at\n', robot.placement(qopt, 6))


         Current function value: 0.000000
         Iterations: 66
         Function evaluations: 712
         Gradient evaluations: 100
The robot finally reached effector placement at
   R =
           1 -4.42656e-09 -4.78269e-09
-2.49232e-10     0.707388    -0.706825
 6.51202e-09     0.706825     0.707388
  p = -0.5  0.1  0.2



/home/cpene/Documents/supaero2025/.venv/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1330: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


### Optimizing in the quaternion space

Let's now work with a floating robot: the quadruped ANYmal. This robot has 12 joints, but Q-space of size 19 (robot.model.nq) and Q-tangent space of size 18 (robot.model.nv). This is because with need 7D vector to encode the robot placement in space, which indeed to only 6 DOF.


In [125]:
robot = robex.load('solo12')
viz = MeshcatVisualizer(robot)
#viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/


In [130]:
viz.display(robot.q0)


Run the following code. Can you explain what just happened? Then correct it to have a proper optimization of ANYmal configuration.

In [115]:
from scipy.optimize import fmin_bfgs, fmin_slsqp

# Récupérer les identifiants numériques (indices) des pattes pour des calculs rapides
# C'est plus efficace que de chercher par nom ('HR_FOOT') à chaque fois
FRAME_NAMES = ['HR_FOOT', 'HL_FOOT', 'FR_FOOT', 'FL_FOOT']
robot.feetIndexes = [robot.model.getFrameId(frameName) for frameName in FRAME_NAMES]



# Ajouter des sphères dans le visualiseur pour représenter les cibles et les positions actuelles des pattes
colors = ['red', 'blue', 'green', 'magenta']
for i, color in enumerate(colors):
    # Sphère pour la position ACTUELLE de la patte i
    viz.addSphere(f"world/{FRAME_NAMES[i]}", 0.05, color)
    # Sphère pour la position DÉSIRÉE (cible) de la patte i
    viz.addSphere(f"world/{FRAME_NAMES[i]}_des", 0.05, color)

targets = [
    np.array([-0.3, -0.15, 0.2]), # Cible pour la patte HR (arrière-droite)
    np.array([-0.3,  0.15, 0.0]), # Cible pour la patte HL (arrière-gauche)
    np.array([ 0.3, -0.15, 0.0]), # Cible pour la patte FR (avant-droite)
    np.array([ 0.3,  0.15, 0.0])  # Cible pour la patte FL (avant-gauche)
]

# On prend la hauteur initiale du torse comme notre objectif
q0 = robot.q0
# L'ID du corps de base est 1 (le monde est 0)
TARGET_HEIGHT = robot.placement(q0, 1).translation[2] 
print(f"Hauteur cible du torse définie à : {TARGET_HEIGHT:.3f} m")


def cost(q):
    legs_cost = 0.0
    for i in range(4):
        p_i = robot.framePlacement(q, robot.feetIndexes[i]).translation
        legs_cost += norm(p_i - targets[i])**2
    return legs_cost

def orientation_constraint(q):
    
    # On récupère la pose complète du torse
    base_pose = robot.placement(q, 1)
    
    # --- Contrainte d'orientation ---
    base_rotation = base_pose.rotation
    rpy = pin.rpy.matrixToRpy(base_rotation)
    orientation_constraints = rpy[:2]  # [roll, pitch]

    # --- Contrainte de hauteur (éviter qu'il reste applatit au sol ---
    current_height = base_pose.translation[2] # On prend la coordonnée Z
    height_constraint = current_height - TARGET_HEIGHT

    return np.array([
        orientation_constraints[0],  # Roulis
        orientation_constraints[1],  # Tangage
        height_constraint           # Erreur de hauteur
    ])
    

def callback(q):

    # Met à jour la position des sphères "actuelles" pour qu'elles suivent les pattes
    for i in range(4):
        p_i = robot.framePlacement(q, robot.feetIndexes[i]).translation
        pose_actuelle = pin.SE3(np.eye(3), p_i)
        viz.applyConfiguration(f'world/{FRAME_NAMES[i]}', pose_actuelle)

    # Affiche la nouvelle configuration du robot
    viz.display(q)
    time.sleep(1)  # Petite pause pour que l'animation soit visible


for i in range(4):
    target_pose = pin.SE3(np.eye(3), targets[i]) # Crée une pose (SE3) à partir de la position cible
    viz.applyConfiguration(f'world/{FRAME_NAMES[i]}_des', target_pose)

#q_optimal = fmin_bfgs(cost, robot.q0, callback=callback)

with_bfgs = 1
if with_bfgs:
    q_optimal = fmin_bfgs(cost, robot.q0, callback=callback)
else:
    q_optimal = fmin_slsqp(
        cost, robot.q0, callback=callback, f_eqcons=orientation_constraint, iprint=2, full_output=1
    )[0]
print("\n *** q_optimal = %s\n\n\n\n" % q_optimal)




Hauteur cible du torse définie à : 0.235 m
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 32
         Function evaluations: 700
         Gradient evaluations: 35

 *** q_optimal = [ 1.61127515e-02  3.42788364e-02  1.89762865e-01 -1.37108744e-01
  7.67170331e-02 -1.58459687e-03  9.59899782e-01  9.51344732e-02
  2.74324442e-01 -2.09284879e+00  7.88347242e-02  2.46581610e-01
 -1.63723720e+00  9.99707530e-02 -3.67273014e-01  1.58508035e+00
 -2.95446984e-01 -2.33655461e-01  2.10780200e+00]






In [132]:
from scipy.optimize import fmin_bfgs, fmin_slsqp

# Récupérer les identifiants numériques (indices) des pattes pour des calculs rapides
# C'est plus efficace que de chercher par nom ('HR_FOOT') à chaque fois
FRAME_NAMES = ['HR_FOOT', 'HL_FOOT', 'FR_FOOT', 'FL_FOOT']
robot.feetIndexes = [robot.model.getFrameId(frameName) for frameName in FRAME_NAMES]



# Ajouter des sphères dans le visualiseur pour représenter les cibles et les positions actuelles des pattes
colors = ['red', 'blue', 'green', 'magenta']
for i, color in enumerate(colors):
    # Sphère pour la position ACTUELLE de la patte i
    viz.addSphere(f"world/{FRAME_NAMES[i]}", 0.05, color)
    # Sphère pour la position DÉSIRÉE (cible) de la patte i
    viz.addSphere(f"world/{FRAME_NAMES[i]}_des", 0.05, color)

targets = [
    np.array([-0.3, -0.15, 0.2]), # Cible pour la patte HR (arrière-droite)
    np.array([-0.3,  0.15, 0.0]), # Cible pour la patte HL (arrière-gauche)
    np.array([ 0.3, -0.15, 0.0]), # Cible pour la patte FR (avant-droite)
    np.array([ 0.3,  0.15, 0.0])  # Cible pour la patte FL (avant-gauche)
]

for i in range(4):
    target_pose = pin.SE3(np.eye(3), targets[i]) # Crée une pose (SE3) à partir de la position cible
    viz.applyConfiguration(f'world/{FRAME_NAMES[i]}_des', target_pose)

# --- Cibles et constantes ---
q0 = robot.q0
viz.display(robot.q0)
pin.forwardKinematics(robot.model, robot.data, q0) #compute 

# L'ID du joint de base est 1
TARGET_HEIGHT = robot.data.oMi[1].translation[2]
print(f"Hauteur cible du torse définie à : {TARGET_HEIGHT:.3f} m")


# --- Fonctions d'Optimisation ---

def cost(q):
    """Calcule le coût : la somme des distances au carré entre les pieds et les cibles."""
    # Met à jour TOUTES les positions des articulations et des frames pour la configuration q
    pin.forwardKinematics(robot.model, robot.data, q)
    pin.updateFramePlacements(robot.model, robot.data)
    
    legs_cost = 0.0
    for i in range(4):
        # Lit la position du pied (frame) directement depuis robot.data (rapide)
        foot_pos = robot.data.oMf[robot.feetIndexes[i]].translation
        legs_cost += norm(foot_pos - targets[i])**2
        
    return legs_cost

def robot_pose_constraint(q):
    """Calcule les erreurs de contrainte (qui doivent être nulles)."""
    # S'assure que les données cinématiques sont à jour pour la configuration q
    pin.forwardKinematics(robot.model, robot.data, q)
    
    # Lit la pose du torse (joint 1) directement depuis robot.data (rapide)
    base_pose = robot.data.oMi[1]
    
    # Contrainte d'orientation (Roulis et Tangage nuls)
    rpy = pin.rpy.matrixToRpy(base_pose.rotation)
    orientation_error = rpy[:2]  # [roll_error, pitch_error]
    
    # Contrainte de hauteur
    height_error = base_pose.translation[2] - TARGET_HEIGHT
    
    return np.array([orientation_error[0], orientation_error[1], height_error])

def callback(q):
    """Callback pour la visualisation."""
    # On doit mettre à jour les placements pour le 'q' actuel fourni par l'optimiseur
    pin.forwardKinematics(robot.model, robot.data, q)
    pin.updateFramePlacements(robot.model, robot.data)
    
    # Affiche le robot
    viz.display(q)
    
    # Met à jour les sphères aux positions actuelles des pieds
    for i in range(4):
        foot_pos = robot.data.oMf[robot.feetIndexes[i]].translation
        viz.applyConfiguration(f'world/{FRAME_NAMES[i]}', pin.SE3(np.eye(3), foot_pos))
        
    time.sleep(5e-1)

# --- Lancement de l'Optimisation ---
print("Lancement de l'optimisation avec la structure de code correcte...")
q_optimal = fmin_slsqp(
    func=cost,
    x0=q0,
    f_eqcons=robot_pose_constraint,
    callback=callback
)
print("Optimisation terminée.")


Hauteur cible du torse définie à : 0.235 m
Lancement de l'optimisation avec la structure de code correcte...
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1702524152064885e-06
            Iterations: 25
            Function evaluations: 504
            Gradient evaluations: 25
Optimisation terminée.


## Configuration of parallel robots
A parallel robot is composed of several kinematic chains (called the robot legs) that are all attached to the same end effector. This imposes strict constraints in the configuration space of the robot: a configuration is valide iff all the legs meets the same end-effector placement. We consider here only the geometry aspect of parallel robots (additionnally, some joints are not actuated, which causes additional problems).

The kinematic structure of a paralel robot indeed induces loops in the joint connection graph. In Pinocchio, we can only represents (one of) the underlying kinematic tree. The loop constraints have to be handled separately. An example that loads 4 manipulator arms is given below.

In [133]:
from supaero2025. load_ur5_parallel import load_ur5_parallel  # noqa: E402

robot = load_ur5_parallel()

In [134]:
viz = MeshcatVisualizer(robot)
#viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/


In [135]:
viz.display(robot.q0)

In [136]:
[w, h, d] = [0.5, 0.5, 0.005]
color = [red, green, blue, transparency] = [1, 1, 0.78, .8]
viz.addBox('world/robot0/toolplate', [w, h, d], color)
Mtool = pin.SE3(pin.utils.rotate('z', 1.268), np.array([0, 0, .75]))
viz.applyConfiguration('world/robot0/toolplate', Mtool)

The 4 legs of the robot are loaded in a single robot model. The 4 effector placements are computed by:

In [137]:
effIdxs = [robot.model.getFrameId('tool0_#%d' % i) for i in range(4)]
robot.framePlacement(robot.q0, effIdxs[0])

SE3(array([[-7.07106781e-01, -7.07106781e-01, -3.46244649e-12,  3.06121952e-01],[ 7.07106781e-01, -7.07106781e-01, -3.46235989e-12,  1.60483362e-01],[-6.12323400e-17, -4.89663865e-12,  1.00000000e+00,  7.49342017e-01],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]))

The loop constraints are that the relative placement of every leg end-effector must stay the same that in the initial configuration given as example in with the configuration *robot.q0* and the plate placement *Mtool*. To be valid, a configuration *q* must satisfy these 4 relative placement constraints.

Consider now that the orientation of the tool plate is given by the following quaternion, with the translation that you like (see [the notebook about rotations if you need more details](appendix1_quaternions.ipynb)): 



In [122]:
quat = pin.Quaternion(0.7, 0.2, 0.2, 0.6).normalized()
print(quat.matrix())

[[ 0.13978495 -0.8172043   0.55913978]
 [ 0.98924731  0.13978495 -0.04301075]
 [-0.04301075  0.55913978  0.82795699]]


**Find using the above optimization routines the configuration of each robot leg so that the loop constraints are all met** for the new orientation of the plate.

In [139]:
import gepetuto.magic
import time
import math
import numpy as np
from numpy.linalg import norm
import pinocchio as pin
import example_robot_data as robex
from scipy.optimize import fmin_bfgs

from supaero2025. load_ur5_parallel import load_ur5_parallel  # noqa: E402

robot = load_ur5_parallel()

viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/


In [172]:
# Positions initiales des éléments 
q0 = robot.q0
viz.display(robot.q0)

# On met à jour les données du robot
pin.forwardKinematics(robot.model, robot.data, q0)
pin.updateFramePlacements(robot.model, robot.data)

# Définition et placement d'une plaque ("toolplate")
[w, h, d] = [0.5, 0.5, 0.005] # width, height, depth
color = [1, 1, 0.78, 0.8] # R, G, B, Alpha (transparence)
viz.addBox('world/robot0/toolplate', [w, h, d], color)

# Définition de la pose initiale de la plaque (posée sur les 4 effecteurs)
Mtool_orig = pin.SE3(pin.utils.rotate('z', 1.268), np.array([0, 0, .75]))
viz.applyConfiguration('world/robot0/toolplate', Mtool_orig)


In [173]:
# IDs des effecteurs (les 4 "mains" du robot)
effIdxs = [robot.model.getFrameId('tool0_#%d' % i) for i in range(4)]

# Le quaternion donné pour la nouvelle orientation
new_orientation = pin.Quaternion(0.7, 0.2, 0.2, 0.6).normalized()
# On choisit une nouvelle translation (par exemple, un peu plus haut et vers l'avant)
new_translation = np.array([0.4, 0.0, 0.6])

# Nouvelle pose cible pour la plaque
o_M_tool_new = pin.SE3(new_orientation.toRotationMatrix(), new_translation)

# Visualisation de la nouvelle plaque cible pour voir où on doit aller
viz.addBox('world/TARGET_PLATE', [0.5, 0.5, 0.005], [0.1, 0.8, 0.2, 0.5]) # Vert transparent
viz.applyConfiguration('world/TARGET_PLATE', o_M_tool_new)



In [174]:
# On calcule la pose de chaque effecteur par rapport à la plaque (On voudra garder ces données constantes)
tool_M_effs_target = []
for idx in effIdxs:
    o_M_eff = robot.data.oMf[idx]
    # La relation est tool_M_eff = (o_M_tool)⁻¹ * o_M_eff
    tool_M_eff = Mtool_orig.inverse() * o_M_eff
    tool_M_effs_target.append(tool_M_eff)

def cost(q):
    """
    Calcule l'erreur totale pour une configuration q.
    L'erreur est la somme des distances (au carré) entre les positions
    actuelles des effecteurs et leurs positions cibles.
    """
    # ÉTAPE CRUCIALE: Mettre à jour la géométrie du robot pour la config 'q' testée
    pin.forwardKinematics(robot.model, robot.data, q)
    pin.updateFramePlacements(robot.model, robot.data)
    
    total_error = 0.0
    
    for i in range(len(effIdxs)):
        # La pose cible de l'effecteur i dans le monde
        o_M_eff_target = o_M_tool_new * tool_M_effs_target[i]
        
        # La pose actuelle de l'effecteur i dans le monde
        o_M_eff_current = robot.data.oMf[effIdxs[i]]
        
        # On calcule l'erreur entre la pose actuelle et la pose cible.
        # Pinocchio propose un outil parfait pour cela: pin.log
        # Il transforme une SE3 (l'erreur de pose) en un vecteur 6D.
        error_transform = o_M_eff_current.inverse() * o_M_eff_target
        error_vector = pin.log(error_transform).vector
        
        # On ajoute le carré de la norme de ce vecteur d'erreur au coût total
        total_error += np.sum(error_vector**2)
        
    return total_error

def callback(q):
    # 1. Mettre à jour les données du robot pour la configuration 'q'
    pin.forwardKinematics(robot.model, robot.data, q)np.dot(np.linalg.pinv(o_Jtool3), o_Jtool3) 
    pin.updateFramePlacements(robot.model, robot.data)
    
    # 2. Afficher le robot lui-même
    viz.display(q)
    
    # 3. Calculer où se trouve la plaque en se basant sur la position actuelle d'un effecteur
    o_M_eff0_current = robot.data.oMf[effIdxs[0]]
    tool_M_eff0 = tool_M_effs_target[0]
    
    # On reconstruit la pose de la plaque à partir de la pose d'un effecteur
    o_M_tool_current = o_M_eff0_current * tool_M_eff0.inverse()
    
    # On met à jour la position de notre unique plaque dans le visualiseur
    viz.applyConfiguration('world/robot0/toolplate', o_M_tool_current)
    
    time.sleep(0.05)
    

q_solution = fmin_bfgs(cost, q0, callback=callback)
#viz.display(q_solution)


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 48
         Function evaluations: 1325
         Gradient evaluations: 53
